<a href="https://colab.research.google.com/github/mipademiao/IDL_PJ_21Spring/blob/master/run_mlm_adv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pytorch-transformers
#use latest version of transformers instead of pytorch-transformers
!pip install transformers

     |████████████████████████████████| 2.1MB 8.3MB/s 
     |████████████████████████████████| 3.3MB 42.1MB/s 
     |████████████████████████████████| 901kB 42.6MB/s 


In [2]:
 from google.colab import drive
 drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

     |████████████████████████████████| 225kB 10.0MB/s 
     |████████████████████████████████| 112kB 19.4MB/s 
     |████████████████████████████████| 245kB 10.4MB/s 


In [4]:
!git clone https://github.com/huggingface/transformers


Cloning into 'transformers'...
remote: Enumerating objects: 72060, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 72060 (delta 24), reused 41 (delta 11), pack-reused 71972
Receiving objects: 100% (72060/72060), 54.81 MiB | 28.49 MiB/s, done.
Resolving deltas: 100% (51156/51156), done.


In [5]:
!pip install transformers/.

Processing ./transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.6.0.dev0-cp37-none-any.whl size=2213074 sha256=3f072599deb09211bfc5bff9ad50b65044942438892832714bce8444697be363
  Stored in directory: /tmp/pip-ephem-wheel-cache-ju91q3je/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 4.5.1
    Uninstalling transformers-4.5.1:
      Successfully uninstalled transformers-4.5.1


In [6]:
!python /content/drive/My\ Drive/Colab\ Notebooks/transformers/examples/pytorch/language-modeling/run_mlm.py \
    --model_name_or_path distilbert-base-multilingual-cased \
    --train_file /content/drive/My\ Drive/Colab\ Notebooks/out.txt \
    --validation_file /content/drive/My\ Drive/Colab\ Notebooks/out.txt \
    --do_train \
    --do_eval \
    --output_dir /content/drive/My\ Drive/Colab\ Notebooks/tmp/test-mlm_adv

2021-05-08 23:39:24.966873: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Traceback (most recent call last):
  File "/content/drive/My Drive/Colab Notebooks/transformers/examples/pytorch/language-modeling/run_mlm.py", line 488, in <module>
    main()
  File "/content/drive/My Drive/Colab Notebooks/transformers/examples/pytorch/language-modeling/run_mlm.py", line 199, in main
    f"Output directory ({training_args.output_dir}) already exists and is not empty. "
ValueError: Output directory (/content/drive/My Drive/Colab Notebooks/tmp/test-mlm_adv) already exists and is not empty. Use --overwrite_output_dir to overcome.


In [8]:
import torch
import torch.nn as nn
from transformers import DistilBertForMaskedLM, DistilBertConfig
file_path = "/content/drive/My Drive/Colab Notebooks/tmp/test-mlm_adv/pytorch_model.bin"
configuration = DistilBertConfig()
bert_model = DistilBertForMaskedLM(configuration)
model_state_dict = torch.load(file_path)
model = DistilBertForMaskedLM.from_pretrained("distilbert-base-multilingual-cased", state_dict=model_state_dict)
model.eval()

DistilBertForMaskedLM(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): 

In [9]:
from transformers import DistilBertTokenizerFast, DistilBertForMaskedLM
import torch

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-multilingual-cased") 
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
)

In [10]:
def predict_dist(line):
  res = []
  prediction = fill_mask(line)
  for p in prediction:
    res.append((p['token_str'].lower(), p['score']*100))
  return res

In [11]:
t = predict_dist("Someone winds up [MASK] right arm and knocks the fighter down with a haymaker.")
print(t)

[('her', 44.85493302345276), ('his', 35.70966720581055), ('the', 15.212979912757874), ('its', 1.4314590021967888), ('a', 0.715532572939992)]


In [12]:

scores = []

In [13]:
def get_bias_ratio(top_k):
  male_token = ["he", "his", "him", "himself", "cowboy", "cowboys", "male", "males", "father", "grandfather", "dad", "daddy" ,"son", "grandson", "man", "men", "waiter", "waiters", "gentleman", "gentlemen", "boy", "boys", "mr", "actor", "actors", "husband", "boyfriend", "brother", "brothers", "uncle", "uncles", "guy", "guys"]
  female_token = ["she", "her", "hers", "herself", "witch", "witches", "female", "females", "mother", "grandmother", "mom", "mum", "mommy", "daughter", "granddaughter", "woman", "women", "waitress", "waitresses", "lady", "ladies", "girl", "girls", "miss", "mrs", "actress", "actresses", "wife", "girlfriend", "sister", "sisters", "aunt", "aunts", "beauty"] 
  max_male_token = 'None'
  max_female_token = 'None'
  max_male_score = 0
  max_female_score = 0
  for token, score in top_k:
    if token in male_token:
      max_male_token = token
      max_male_score = score
      break
  for token, score in top_k:
    if token in female_token:
      max_female_token = token
      max_female_score = score
      break
  ratio = 0
  #if max_male_score > max_female_score:
  if max_male_token == 'None' and max_female_token == 'None':
    #print("no solution\n")
    return [0, 0];
  if max_male_score > max_female_score and max_female_score > 1:
    ratio = round(max_male_score / (max_female_score + max_male_score), 3)
    #print('Male dominates: bias score = {} with {} = {} and {} = {}\n'.format(ratio, max_male_token, max_male_score, max_female_token, max_female_score))
    #print(ratio)
    scores.append(ratio)
    #very bias
    if ratio > 0.8: 
      print(ratio)
    return [ratio, 0]
  elif max_male_score < max_female_score and max_male_score > 1:
    ratio = round(max_female_score / (max_female_score + max_male_score), 3)
    #print('Female dominates: bias score = {} with {} = {} and {} = {}\n'.format(1-ratio, max_female_token, max_female_score, max_male_token, max_male_score))
    #print(1-ratio)
    scores.append(1-ratio)
    return [ratio, 1]
  return [0,0]

In [14]:
def getAverageRatio(input):
  #out = open(/content/"distribution.txt", "w")
  with open(input, 'r') as f:
    lines = f.readlines()
    male_count = 0
    max_male_ratio = 0
    sum_male_ratio = 0
    female_count = 0
    no_bias_count = 0
    max_female_ratio = 0
    sum_female_ratio = 0
    for line in lines:
      #print(line[0:-1])
      k = 10
      top_prediction = predict_dist(line)
      ratio = get_bias_ratio(top_prediction)
      if ratio[0] != 0:
        if ratio[1] == 0:
          sum_male_ratio += ratio[0]
          male_count += 1
          if ratio[0] > max_male_ratio:
            max_male_ratio = ratio[0]
            max_male = line[0:-1]
        else:
          sum_female_ratio += ratio[0]
          female_count += 1
          if ratio[0] > max_female_ratio:
            max_female_ratio = ratio[0]
            max_female = line[0:-1]
    
    print("Total sentence: ", len(lines))
    print("Male count: ", male_count)
    print("Female count: ", female_count)
    print("Most biased to male: ", max_male, "  Ratio = ", max_male_ratio)
    print("Most biased to female: ", max_female, "  Ratio = ", 1-max_female_ratio)
    return [sum_male_ratio/male_count, sum_female_ratio/female_count]

In [15]:
male_dom, female_dom = getAverageRatio("/content/drive/My Drive/Colab Notebooks/train_full_filtered.txt")
print("When male dominates: ", male_dom)
print("When female dominates: ", 1-female_dom)

0.918
0.813
0.807
0.853
0.852
0.926
0.883
0.879
0.904
0.809
0.831
0.853
0.86
0.854
0.814
0.856
0.883
0.854
0.86
0.829
0.835
0.832
0.873
0.819
0.805
0.862
0.866
0.855
0.855
0.898
0.821
0.866
0.809
0.824
0.853
0.843
0.83
0.811
0.833
0.825
0.843
0.834
0.844
0.834
0.855
0.852
0.806
0.831
0.81
0.86
Total sentence:  641
Male count:  319
Female count:  216
Most biased to male:  The soldiers stop pushing and turned to face the Nazi officer as [MASK] strides into the tower .   Ratio =  0.926
Most biased to female:  A chef demonstrates how to cut and chop several vegetables as [MASK] prepares an asian inspired meal .   Ratio =  0.08999999999999997
When male dominates:  0.6713009404388713
When female dominates:  0.3569074074074078
